In [94]:

import numpy as np
import math
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,Aer, transpile, execute
from qiskit.visualization import plot_histogram

In [95]:
def create_lists(clause):
    pos_var = []
    neg_var = []
    none = []
    for index,item in enumerate(clause):
        if item>0:
            pos_var.append(index)
        elif item<0:
            neg_var.append(index)
        else:
             none.append(index)
    return pos_var,neg_var,none

In [96]:
def encode_clause(clause,circuit,qreg):
    
    num_of_variables = len(clause)
    positive_var,negative_var,either_var = create_lists(clause)
    
    #Building the oracle
    
    #negating the variables
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()

    #Logical OR operation
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    #qc.mct([qreg[i] for i in range(num_of_variables)],qr[[-1]])
    circuit.mct([qreg[i] for i in (positive_var+negative_var)],qreg[-1])
    circuit.x([qreg[i] for i in (positive_var+negative_var)])
    circuit.barrier()

    #Bringing the negated variables to original form
    if len(negative_var)!=0:
        circuit.x(qreg[i] for i in negative_var)
        circuit.barrier()
    
    return circuit

In [97]:
def diffusion_circ(list):
    num_of_variables = len(list)
    
    qr = QuantumRegister(num_of_variables+1)
    cr = ClassicalRegister(num_of_variables+1)
    qc = QuantumCircuit(qr,cr)
    
    qc.h(qr[:])
    qc.x(qr[:])
    qc.h(qr[-1])
    qc.mct([qr[i] for i in range(num_of_variables)],qr[[-1]])
    qc.h(qr[-1])
    qc.x(qr[:])
    qc.h(qr[:])
    
    return qc

### 3-SAT problem

In [106]:
def hybrid_3mct(circuit,clause,aux,f_in,f_out):
    # circuit.ccx(control_qbits[0],control_qbits[1],aux[0])
    # circuit.ccx(control_qbits[2],aux[0],f_out[0])
    # circuit.ccx(control_qbits[0],control_qbits[1],aux[0])
    control_qbits = [abs(val) for i,val in enumerate(clause) if val!=0]
    print(control_qbits)
    circuit.ccx(f_in[control_qbits[0]-1],f_in[control_qbits[1]-1],aux[0])
    circuit.ccx(f_in[control_qbits[2]-1],aux[0],f_out[0])
    circuit.ccx(f_in[control_qbits[0]-1],f_in[control_qbits[1]-1],aux[0])
    return circuit

def hybrid_oracle(circuit,f_in,f_out,aux,n,sat_formula):
    
    for clause in sat_formula:    
        for i in range(len(clause)):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[clause[i]-1])
        hybrid_3mct(circuit,clause,aux,f_in,f_out)
        for i in range(len(clause)):
            #Encoding positive variables
            if clause[i]>0:
                circuit.x(f_in[clause[i]-1])
        circuit.barrier()
        
    return circuit  

def diffusion_circ3(circuit,f_in,aux):
    circuit.h(f_in[:])
    circuit.x(f_in[:])
    circuit.h(f_in[-1])
    if len(f_in[:])==3:
        circuit.ccx(f_in[0],f_in[1],f_in[-1])
    elif len(f_in[:])==4:
        circuit.ccx(f_in[0],f_in[1],aux[0])
        circuit.ccx(aux[0],f_in[2],f_in[-1])
        circuit.ccx(f_in[0],f_in[1],aux[0])
    circuit.h(f_in[-1])
    circuit.x(f_in[:])
    circuit.h(f_in[:])
    circuit.barrier()
    
    return circuit

In [108]:
def hybrid_sat3(sat_formula):
    n = max(max(sat_formula))
    print(n)
    f_in = QuantumRegister(n)
    f_out = QuantumRegister(1)
    aux = QuantumRegister(n-2)
    cr = ClassicalRegister(n)
    circuit = QuantumCircuit(f_in,f_out,aux,cr)

    #Superposition of input states
    for i in range(n):
        circuit.h(f_in[i])
    # circuit.x(aux[1])
    # circuit.h(aux[1])
    #initializing f_out qubit with state |1>
    circuit.x(f_out[0])
    circuit.barrier()
    
    #oracle
    for _ in range(math.floor(math.pi*math.sqrt(2**n)/4 - 0.5)):
        hybrid_oracle(circuit,f_in,f_out,aux,n,sat_formula)
        circuit.z(f_out[0])
    # circuit.cx(f_out[0],aux[1])
        circuit.barrier()
    #diffusion operator
        diffusion_circ3(circuit,f_in,aux)
    #phase kickback

    circuit.measure(f_in[:],cr[:])
    
    return circuit
        

In [57]:
#Generating random k-SAT problem
import random
def random_kcnf(n_literals, n_conjuncts, k=3):
    result = []
    for _ in range(n_conjuncts):
        sat_clause = random.sample(range(1,n_literals+1),3)
        k = random.randint(0,4)
        if k==1:
            pick_var = random.randint(0,3)
            sat_clause[pick_var] = -sat_clause[pick_var]
        elif k==2:
            pick_var = random.sample(range(0,3),2)
            sat_clause[pick_var[0]] = -sat_clause[pick_var[0]]
            sat_clause[pick_var[1]] = -sat_clause[pick_var[1]]
        elif k==3:
            sat_clause = [-x for x in sat_clause]
        elif k==0:
            sat_clause = sat_clause
        #print(sat_clause)
        sat_clause = sorted(sat_clause,key=abs) 
        result.append(sat_clause)
    return result

In [58]:
random_kcnf(3,6,3)

[[1, -2, -3], [-1, -2, 3], [-1, -2, -3], [1, 2, 3], [-1, -2, -3], [1, 2, 3]]

In [41]:
def classical_sat_checker(sat_formula,quantum_solution):
    
    sat_solution = []
    for k in range(len(quantum_solution)):              #for each clause in a classical part of sat_formula
        final_result = []
        for sat_clause in sat_formula:                  #for each solution from set of quantum valid solutions
            clause_result = []
            for variable,quant_variable in zip(sat_clause,list(quantum_solution[k])):   #For a each sat clause and each quantum solution we zip each literals correspondingly
                if variable>0 and int(quant_variable)>0:
                    clause_result.append('yes')
                elif variable<0 and int(quant_variable)==0:
                    clause_result.append('yes')
                else:
                    clause_result.append('no')
            # print(clause_result)
            if 'yes' in clause_result:
                final_result.append('yes')
            else:
                final_result.append('no')
        all_yes = all(element == 'yes' for element in final_result)
        #print(final_result)
        if all_yes:
            sat_solution.append(quantum_solution[k])                     #Assuming we have only one sat solution

    if len(sat_solution)==0:
        sat_solution.append('No solution exists')
    return(sat_solution)
